In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import numpy as np

In [4]:
Tweets = pd.read_csv("Tweets.csv")
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [13]:
Tweets.groupby(['airline_sentiment']).size()

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [14]:
Tweets = Tweets[Tweets['airline_sentiment_confidence']> 0.8]

In [15]:
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [16]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [17]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment'])
print(y)

[1 0 0 ... 0 1 0]


In [18]:
print(X)

[[97 62  0 ...  0  0  0]
 [97 99  1 ...  0  0  0]
 [97  9 99 ...  0  0  0]
 ...
 [13 98 93 ...  0  0  0]
 [13 89  1 ...  0  0  0]
 [13  6 23 ...  0  0  0]]


In [19]:
y = np_utils.to_categorical(y)
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)
X_test

array([[16, 11, 14, ...,  0,  0,  0],
       [16, 44,  6, ...,  0,  0,  0],
       [16, 51, 26, ...,  0,  0,  0],
       ...,
       [13,  3, 27, ...,  0,  0,  0],
       [16, 42, 23, ...,  0,  0,  0],
       [ 8, 11, 78, ...,  0,  0,  0]])

In [21]:
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [22]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(modelo.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1638656   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 1,894,047
Trainable params: 1,894,047
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
modelo.fit(X_train, y_train, epochs=10, batch_size=30,verbose=True,validation_data=(X_test, y_test))

Epoch 1/10
245/245 [==============================] - 90s 339ms/step - loss: 0.8150 - accuracy: 0.7069 - val_loss: 0.8174 - val_accuracy: 0.7046
Epoch 2/10
245/245 [==============================] - 78s 320ms/step - loss: 0.8099 - accuracy: 0.7077 - val_loss: 0.8142 - val_accuracy: 0.7046
Epoch 3/10
245/245 [==============================] - 77s 313ms/step - loss: 0.8110 - accuracy: 0.7077 - val_loss: 0.8117 - val_accuracy: 0.7046
Epoch 4/10
245/245 [==============================] - 102s 415ms/step - loss: 0.8095 - accuracy: 0.7077 - val_loss: 0.8124 - val_accuracy: 0.7046
Epoch 5/10
245/245 [==============================] - 97s 396ms/step - loss: 0.8105 - accuracy: 0.7077 - val_loss: 0.8121 - val_accuracy: 0.7046
Epoch 6/10
245/245 [==============================] - 93s 378ms/step - loss: 0.8088 - accuracy: 0.7077 - val_loss: 0.8127 - val_accuracy: 0.7046
Epoch 7/10
245/245 [==============================] - 46s 187ms/step - loss: 0.8089 - accuracy: 0.7077 - val_loss: 0.8123 - val_a

In [24]:
loss, accuracy = modelo.evaluate(X_test,y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

99/99 [==============================] - 4s 39ms/step - loss: 0.8131 - accuracy: 0.7046
Loss:  0.8131201267242432
Accuracy:  0.7045888900756836


In [25]:
prev = modelo.predict(X_test)
print(prev)

99/99 [==============================] - 6s 39ms/step
[[0.6909049  0.14303313 0.16606195]
 [0.6909049  0.14303313 0.16606198]
 [0.6909049  0.14303312 0.16606195]
 ...
 [0.6909049  0.14303315 0.16606198]
 [0.6909049  0.14303315 0.16606198]
 [0.6909049  0.14303315 0.16606198]]
